In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

### Extracting features of 2020 movies from Wikipedia


In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"


In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
type(tables[0])


bs4.element.Tag

In [7]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [8]:
dfs = [df1, df2, df3, df4]
df = pd.concat(dfs, ignore_index=True)

In [9]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,J A N U A R Y,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2],NaN
1,J A N U A R Y,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3],NaN
2,J A N U A R Y,10,Like a Boss,Paramount Pictures / Artists First,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4],NaN
3,J A N U A R Y,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN,NaN
4,J A N U A R Y,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5],NaN
...,...,...,...,...,...,...,...
271,D E C E M B E R,25,We Can Be Heroes,Netflix / Troublemaker Studios / Double R Prod...,Robert Rodriguez (director/screenplay); Priyan...,NaN,[242]
272,D E C E M B E R,25,News of the World,Universal Pictures / Perfect World Pictures,Paul Greengrass (director/screenplay); Luke Da...,NaN,[243]
273,D E C E M B E R,25,One Night in Miami...,Amazon Studios,Regina King (director); Kemp Powers (screenpla...,NaN,[244]
274,D E C E M B E R,25,Promising Young Woman,Focus Features / FilmNation Entertainment,Emerald Fennell (director/screenplay); Carey M...,NaN,[245]


In [10]:
df_2020 = df[['Title','Cast and crew']]


In [11]:
df_2020

,Title,Cast and crew
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...
1,Underwater,"William Eubank (director); Brian Duffield, Ada..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col..."
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...
...,...,...
271,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...
272,News of the World,Paul Greengrass (director/screenplay); Luke Da...
273,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...
274,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...


In [12]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '539e85067de975ed98423b9d9da95b1f'

In [13]:
from tmdbv3api import Movie
import ast
tmdb_movie = Movie()
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if result:
        movie_id = result[0].id
        response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
        data_json = response.json()
        print(data_json)
        if data_json['genres']:
            genre_str = " " 
            for i in range(0,len(data_json['genres'])):
                genres.append(data_json['genres'][i]['name'])
            genre_list = str(genres)
            return genre_list
        else:
            np.nan
    else:
        np.nan

In [14]:
df_2020['Genre'] = df_2020['Title'].map(lambda x: get_genre(str(x)))


{'adult': False, 'backdrop_path': '/iuW78LWxApCvjLuMebcICwF2owQ.jpg', 'belongs_to_collection': None, 'budget': 10000000, 'genres': [{'id': 27, 'name': 'Horror'}, {'id': 9648, 'name': 'Mystery'}], 'homepage': '', 'id': 465086, 'imdb_id': 'tt3612126', 'original_language': 'en', 'original_title': 'The Grudge', 'overview': 'After a young mother murders her family in her own house, a detective attempts to investigate the mysterious case, only to discover that the house is cursed by a vengeful ghost. Now targeted by the demonic spirits, the detective must do anything to protect herself and her family from harm.', 'popularity': 18.812, 'poster_path': '/vN7JHlHOT9rHNDU27tfYqhABBj5.jpg', 'production_companies': [{'id': 768, 'logo_path': None, 'name': 'Ghost House Pictures', 'origin_country': 'US'}], 'production_countries': [{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso_3166_1': 'US', 'name': 'United States of America'}], 'release_date': '2020-01-01', 'revenue': 42800000, 'runtime': 93, 'spoken_

/var/folders/qv/gqb3583x48g_90k1j3lvlvyr0000gn/T/ipykernel_50098/2640262719.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['Genre'] = df_2020['Title'].map(lambda x: get_genre(str(x)))


In [15]:
df_2020

,Title,Cast and crew,Genre
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,"['Horror', 'Mystery']"
1,Underwater,"William Eubank (director); Brian Duffield, Ada...","['Horror', 'Science Fiction', 'Action', 'Adven..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",['Comedy']
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,['Drama']
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,"['Crime', 'Thriller', 'Drama']"
...,...,...,...
271,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,"['Action', 'Fantasy', 'Family', 'Comedy']"
272,News of the World,Paul Greengrass (director/screenplay); Luke Da...,"['Drama', 'Western', 'Adventure', 'Action']"
273,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,['Drama']
274,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,"['Thriller', 'Crime', 'Drama']"


In [16]:
def list_to_string(x):
    if x:
        return x.replace(',', '').replace('[','').replace(']','').replace("'",'')
    else:
        return np.NaN

In [17]:
df_2020['genres'] = df_2020['Genre'].map(lambda x: list_to_string(x))

/var/folders/qv/gqb3583x48g_90k1j3lvlvyr0000gn/T/ipykernel_50098/368756313.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['genres'] = df_2020['Genre'].map(lambda x: list_to_string(x))


In [18]:
df_2020

,Title,Cast and crew,Genre,genres
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,"['Horror', 'Mystery']",Horror Mystery
1,Underwater,"William Eubank (director); Brian Duffield, Ada...","['Horror', 'Science Fiction', 'Action', 'Adven...",Horror Science Fiction Action Adventure
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",['Comedy'],Comedy
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,['Drama'],Drama
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,"['Crime', 'Thriller', 'Drama']",Crime Thriller Drama
...,...,...,...,...
271,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,"['Action', 'Fantasy', 'Family', 'Comedy']",Action Fantasy Family Comedy
272,News of the World,Paul Greengrass (director/screenplay); Luke Da...,"['Drama', 'Western', 'Adventure', 'Action']",Drama Western Adventure Action
273,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,['Drama'],Drama
274,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,"['Thriller', 'Crime', 'Drama']",Thriller Crime Drama


In [19]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [20]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))


/var/folders/qv/gqb3583x48g_90k1j3lvlvyr0000gn/T/ipykernel_50098/3962101783.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))


In [21]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [22]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

/var/folders/qv/gqb3583x48g_90k1j3lvlvyr0000gn/T/ipykernel_50098/3646981363.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [23]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [24]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [25]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [26]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))


In [27]:
df_2020


,Title,Cast and crew,Genre,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,"['Horror', 'Mystery']",Horror Mystery,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,Underwater,"William Eubank (director); Brian Duffield, Ada...","['Horror', 'Science Fiction', 'Action', 'Adven...",Horror Science Fiction Action Adventure,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",['Comedy'],Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,['Drama'],Drama,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,"['Crime', 'Thriller', 'Drama']",Crime Thriller Drama,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs
...,...,...,...,...,...,...,...,...
271,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,"['Action', 'Fantasy', 'Family', 'Comedy']",Action Fantasy Family Comedy,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin
272,News of the World,Paul Greengrass (director/screenplay); Luke Da...,"['Drama', 'Western', 'Adventure', 'Action']",Drama Western Adventure Action,Paul Greengrass,Tom Hanks,Helena Zengel,NaN
273,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,['Drama'],Drama,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge
274,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,"['Thriller', 'Crime', 'Drama']",Thriller Crime Drama,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie


In [28]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})


In [29]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]


In [30]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery,The Grudge
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Horror Science Fiction Action Adventure,Underwater
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,Like a Boss
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,Three Christs
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,Inherit the Viper
...,...,...,...,...,...,...
271,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy,We Can Be Heroes
272,Paul Greengrass,Tom Hanks,Helena Zengel,NaN,Drama Western Adventure Action,News of the World
273,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,One Night in Miami...
274,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,Promising Young Woman


In [31]:
new_df20['actor_2_name'] = new_df20['actor_2_name'].replace(np.nan, 'unknown')
new_df20['actor_3_name'] = new_df20['actor_3_name'].replace(np.nan, 'unknown')
new_df20['director_name'] = new_df20['director_name'].replace(np.nan, 'unknown')
new_df20['movie_title'] = new_df20['movie_title'].replace(np.nan, 'unknown')
new_df20['actor_1_name'] = new_df20['actor_1_name'].replace(np.nan, 'unknown')
new_df20['genres'] = new_df20['genres'].replace(np.nan, 'unknown')

In [32]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']


In [33]:

new_df20['comb'] = new_df20['comb'].replace(np.nan, 'unknown')


In [34]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

In [35]:
new_df20


,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery,the grudge,Andrea Riseborough Demián Bichir John Cho Nico...
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Horror Science Fiction Action Adventure,underwater,Kristen Stewart Vincent Cassel Jessica Henwick...
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,like a boss,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,three christs,Richard Gere Peter Dinklage Walton Goggins Jon...
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,inherit the viper,Josh Hartnett Margarita Levieva Chandler Riggs...
...,...,...,...,...,...,...,...
271,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy,we can be heroes,Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...
272,Paul Greengrass,Tom Hanks,Helena Zengel,unknown,Drama Western Adventure Action,news of the world,Tom Hanks Helena Zengel unknown Paul Greengras...
273,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,one night in miami...,Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...
274,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,promising young woman,Carey Mulligan Bo Burnham Alison Brie Emerald ...


In [36]:
old_df = pd.read_csv('/Users/rasikagulhane/MovieRecommendationSystem_-_SentimentAnalysis/final_data.csv')
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action|Adventure|Fantasy|Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action|Adventure|Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action|Adventure|Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action|Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5879,"Nick Bruno, Troy Quane",Will Smith,Will Smith,Rashida Jones,Animation Action Adventure Comedy Family,spies in disguise,Will Smith Will Smith Rashida Jones Nick Bruno...
5880,Greta Gerwig,Saoirse Ronan,Saoirse Ronan,Florence Pugh,Drama Romance History,little women,Saoirse Ronan Saoirse Ronan Florence Pugh Gret...
5881,Sam Mendes,George MacKay,George MacKay,Mark Strong,War Drama History,1917,George MacKay George MacKay Mark Strong Sam Me...
5882,Destin Daniel Cretton,Michael B. Jordan,Michael B. Jordan,Brie Larson,Drama Crime History,just mercy,Michael B. Jordan Michael B. Jordan Brie Larso...


In [37]:
old_df.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [38]:
# final_df = old_df.append(new_df20,ignore_index=True)
final_df = pd.concat((old_df,new_df20), ignore_index=True)


In [39]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action|Adventure|Fantasy|Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action|Adventure|Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action|Adventure|Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action|Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6155,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy,we can be heroes,Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...
6156,Paul Greengrass,Tom Hanks,Helena Zengel,unknown,Drama Western Adventure Action,news of the world,Tom Hanks Helena Zengel unknown Paul Greengras...
6157,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,one night in miami...,Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...
6158,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,promising young woman,Carey Mulligan Bo Burnham Alison Brie Emerald ...


In [40]:
final_df.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [ ]:
final_df.to_csv('main_data.csv',index=False)